# Uniview module for of the asteroid explosion simulation 

### From Charles El Mir from [this article](https://www.sciencedirect.com/science/article/pii/S001910351830349X?via%3Dihub).


*Aaron Geller, 2019*

### Imports and function definitions

In [44]:
#This directory contains all the data needed for the module. It should be in the same directory as the notebook
dataFolder = "data" 
import sys, os, shutil, errno, string, urllib


sys.path.append(( os.path.abspath( os.path.join(os.path.realpath("__file__"),  os.pardir, os.pardir) )))
import uvmodlib.v1 as uvmod

### USES Conf Template 

In [55]:
Template = """mesh
{  
    data asteroid    ./modules/$folderName/asteroid.raw 
    dataHints asteroid disablePostprocessing

    data quad    ./modules/$folderName/quad.3ds

    cullRadius $cr
    glslVersion 330
    
    propertyCollection 
    {        
        __objectName__
        {           

            vec1f pSize 300 | public | desc "point size " | widget slider | range 0 1000
            vec1f eventTime 0 | public | desc "time " | widget slider | range 0 130000
            vec1f dMin 0.125 | public | desc "minimum damage"
            vec1f dMax 0.23 | public | desc "maximum damage"
            vec1f alpha 0.8 | public | desc "alpha" | widget slider | range 0 1
            
            vec1f transitionLength 10 | public | desc "transition length in seconds" 
            bool jump true | public  | desc "jump to time without transition" 
            
        }
    }
    ############# to hold the time information
    renderTexture
    {
        name                      stateTexture
        width                     1
        height                    1
        numTextures               1
        isPingPong                true
        isPersistent              true
        isFramePersistent         true
        internalTextureFormat     GL_RGB32F
        magnify                   GL_NEAREST
        minify                    GL_NEAREST
    }
    ############# set Transition State
    pass 
    {
        useDataObject quad
        renderTarget
        {
            name stateTexture
            enableColorClear false
        }
         shader
        {
            type defaultMeshShader
            {
                vertexShader   ./modules/$folderName/pass0.vs
                fragmentShader ./modules/$folderName/state.fs
                textureFBO stateTexture stateTexture
                
                stateManagerVar __objectName__.transitionLength transitionLength 
                stateManagerVar __objectName__.jump jump 
                stateManagerVar __objectName__.eventTime eventTime 
                
                parameter2f timeRange 0 133000
           }
        }      
    }
    
    ############# asteroid
    pass
    {
        useDataObject asteroid
        shader
        {
            type defaultMeshShader
            {
                geometryShader   ./modules/$folderName/asteroid.gs
                vertexShader ./modules/$folderName/asteroid.vs
                fragmentShader   ./modules/$folderName/asteroid.fs

                textureFBO stateTexture stateTexture

                texture cmap ./modules/$folderName/cmap.png
                {             
                    wrapModeS GL_CLAMP_TO_EDGE
                    wrapModeR GL_CLAMP_TO_EDGE
                    #colorspace linear
                }
                
                stateManagerVar __objectName__.pSize  pSize
                stateManagerVar __objectName__.dMin  dMin
                stateManagerVar __objectName__.dMax  dMax
                stateManagerVar __objectName__.alpha  alpha

                glState
                {
                    UV_CULL_FACE_ENABLE false
                    UV_BLEND_ENABLE true
                    UV_DEPTH_ENABLE false
                    UV_WRITE_MASK_DEPTH true
                    UV_BLEND_FUNC GL_SRC_ALPHA GL_ONE_MINUS_SRC_ALPHA
                }
            }

        }    
    }
}"""

### Asteroid class

In [56]:
class asteroid():
    def __init__(self, object):
        self.object = object
        uvmod.Utility.ensurerelativepathexsists("asteroid.gs",dataFolder)
        uvmod.Utility.ensurerelativepathexsists("asteroid.vs",dataFolder)
        uvmod.Utility.ensurerelativepathexsists("asteroid.fs",dataFolder)
        self.cr = 1000
        self.Scale = 1

    def generatemod(self):
        self.object.setgeometry(self.object.name+"Mesh.usesconf")
        return self.object.generatemod()
    def generatefiles(self, absOutDir, relOutDir):
        fileName = self.object.name+"Mesh.usesconf"
        s = string.Template(Template)
        f = open(absOutDir+"\\"+fileName, 'w')
        if f:
            f.write(s.substitute(folderName = relOutDir,
                                 cr = self.cr,
                                 Scale = self.Scale
                                 ))
            f.close()
        uvmod.Utility.copyfoldercontents(os.getcwd()+"\\"+dataFolder, absOutDir)

### Object Instantiation

In [57]:
model = asteroid(uvmod.OrbitalObject())
generator = uvmod.Generator()
scene = uvmod.Scene()
scene.setname("AsteroidExplosion")
scene.setparent("Earth")
scene.setentrydist(100000) #not sure what this does
scene.setunit(1000)
scene.setstaticposition(-1e7,0,0)

modinfo = uvmod.ModuleInformation()

### Specify Settings and generate the module

In [58]:
model.object.setcameraradius(1) #I think smaller values here mean less clipping
model.object.setcoord(scene.name)
model.object.setname("AsteroidExplosion")
model.object.setguiname("/Adler/Universe Update/Asteroid Explosion")
model.object.settargetradius(100) #this is the location where the camera will fly to
model.object.showatstartup(False)
model.cr = 100 #I think this is clipping on the back end?
modinfo.setname("Asteroid Explosion")
modinfo.setauthor("Aaron Geller<sup>1</sup> and Charles El Mir<sup>2</sup>\
                <br />(1) Adler Planetarium,<br />(2)Johns Hopkins University")
modinfo.cleardependencies()
modinfo.setdesc("Uniview module for the asteroid explosion simulation from Charles El Mir, et al. \
                in this article : https://www.sciencedirect.com/science/article/pii/S001910351830349X?via%3Dihub")
#modinfo.setthumbnail("data/R0010133.JPG")
modinfo.setversion("1.0")

generator.generate("AsteroidExplosion",[scene],[model],modinfo)
uvmod.Utility.senduvcommand(model.object.name+".reload")

Unable to connect to Uniview


## Helper Functions for modifing code
*Reload Module and Shaders in Uniview*

In [14]:
uvmod.Utility.senduvcommand(model.object.name+".reload; system.reloadallshaders")

*Copy modified Shader files and reload*

In [15]:
from config import Settings

In [16]:
uvmod.Utility.copyfoldercontents(os.getcwd()+"\\"+dataFolder, Settings.uvcustommodulelocation+'\\'+model.object.name)
uvmod.Utility.senduvcommand(model.object.name+".reload")

### Create colormap texture

In [18]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
%matplotlib inline

In [19]:
gradient = np.linspace(0, 1, 256)
gradient = np.vstack((gradient, gradient))

def plot_cmap(colormap):
    fig=plt.imshow(gradient, aspect=1, cmap=colormap)
    plt.axis('off')
    fig.axes.get_xaxis().set_visible(False)
    fig.axes.get_yaxis().set_visible(False)
    plt.savefig("data/cmap.png", bbox_inches='tight',pad_inches=0)

plot_cmap('YlOrBr_r')